In [27]:
def get_xyxy_from_txt(label):
    gt_boxes = list(map(float, label.split(' ')[1:]))
    x_center = gt_boxes[0]*1024
    y_center = gt_boxes[1]*128
    box_width = gt_boxes[2]*1024
    box_height = gt_boxes[3]*128
    
    x1 = int(x_center - (box_width / 2))
    y1 = int(y_center - (box_height / 2))
    x2 = int(x_center + (box_width / 2))
    y2 = int(y_center + (box_height / 2))

    return x1, y1, x2, y2

In [32]:
import os
import cv2
import math
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from natsort import natsorted
from ouster import client
from ouster import pcap
from ouster.client import Scans, XYZLut, SensorInfo, destagger
from ouster.client.data import LidarScan
from ultralytics import YOLO
from contextlib import closing
from collections import defaultdict


output_dict = {3:["GO AHEAD", (0, 255, 0)], 2:["BE CAREFUL", (0, 155, 100)], 1:["SLOW DOWN", (0, 100, 155)], 
                0:["BREAK", (0, 0, 255)]}

# Load the YOLOv8 model
model = YOLO('../weights/best_3000_s_100.pt')

# Paths to pcap and json files
metadata_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.json"
pcap_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.pcap"

# Making PacketSource from data
with open(metadata_path, 'r') as f:
    metadata = client.SensorInfo(f.read())

fps = int(str(metadata.mode)[-2:])
width = int(str(metadata.mode)[:4])
height = int(str(metadata.prod_line)[5:])

pcap_file = pcap.Pcap(pcap_path, metadata)

xyz_lut = client.XYZLut(metadata) #call cartesian lookup table

# Store the track history
track_history = defaultdict(lambda: [])     # dictionary: {key=id, value=[xyz_val_0, ... ,xyz_val_99]}
box_history = defaultdict(lambda: [])
iou_values = []
groundtruth_iou = []

directory_path = 'groundtruthing_labels/'

with closing(Scans(pcap_file)) as scans:

    save_path = "C:/Users/szyme/Ouster/Dangerous-situations-with-pedastrians/results_mp4"
    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    files = os.listdir(directory_path)
    files_sorted = natsorted(files)

    i = 0
    for scan, filename in zip(scans, files_sorted):
        i += 1

        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            labels_file = f
        with open(labels_file, 'r') as file:
            labels = file.readlines()

        sig_field = scan.field(client.ChanField.SIGNAL)
        sig_destaggered = destagger(metadata, sig_field)
        scaling_factor = 0.004
        scaled_arr = sig_destaggered / (0.5 + scaling_factor * sig_destaggered)
        signal_image = scaled_arr.astype(np.uint8)
        combined_img = np.dstack((signal_image, signal_image, signal_image))

        xyz_destaggered = client.destagger(metadata, xyz_lut(scan))

        results = model.track(source=combined_img, persist=True, imgsz=1024, tracker='../trackers/tracker.yaml', verbose=False)

        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)

        if (results[0].boxes.id == None):
            ids = ''
        else:
            ids = results[0].boxes.id.cpu().numpy().astype(int)

        priority = 3
        distance = 0
        print(len(boxes))
        print(len(ids))
        print(len(labels))
        if len(labels) != len(boxes):
            # print("AAAA")

            for box, id, label in zip(boxes, ids, labels):
                x1, y1, x2, y2 = get_xyxy_from_txt(label)
                print(f"{x1}, {y1}, {x2}, {y2}")

                print(box)
                center_x, center_y = int((box[0] + box[2])/2), int((box[1] + box[3])/2) #

                xyz_val = xyz_destaggered[(center_y, center_x)] #get the (x,y,z) coordinates with the lookup table 

                track = track_history[id] #save the (x,y,z) coordinates for distance calculation
                track.append(xyz_val)

                new_box = box_history[id]
                new_box.append((box[0], box[1], box[2], box[3]))

                cv2.rectangle(combined_img, (box[0], box[1]), (box[2], box[3]), output_dict[priority][1], 2)
                cv2.rectangle(combined_img, (box[0], box[1]+2), (box[0]+20+len(str(id))*11, box[1]-12), (255, 255, 255), -1)
                cv2.putText(combined_img, f"Id {id}", (box[0], box[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        cv2.putText(combined_img, f"{output_dict[priority][0]}", (470, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, output_dict[priority][1], 2)

        if True:
            cv2.imshow("YOLOv8 Tracking", combined_img)
            cv2.waitKey(1)  # 1 millisecond

        vid_writer.write(combined_img)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        if i >= 1000:
            break

    vid_writer.release()
    cv2.destroyAllWindows()

0
0
0
2
2
2
477, 62, 490, 93
[475  61 489  96]
162, 67, 168, 88
[161  67 170  90]
2
2
2
476, 61, 490, 97
[475  61 489  96]
162, 67, 172, 89
[162  66 171  89]
2
2
2
476, 61, 489, 95
[475  61 489  96]
164, 67, 172, 88
[163  67 172  89]
2
2
2
475, 61, 489, 94
[475  61 489  95]
165, 68, 174, 90
[165  67 173  89]
2
2
2
167, 67, 176, 89
[475  61 489  95]
477, 62, 488, 95
[166  66 175  89]
2
2
2
476, 62, 489, 95
[475  61 489  95]
168, 66, 176, 88
[168  66 177  89]
2
2
2
171, 67, 179, 87
[475  61 489  95]
476, 61, 490, 95
[170  66 178  88]
2
2
2
171, 67, 180, 89
[475  61 489  95]
476, 61, 487, 94
[172  67 180  88]
2
2
1
475, 62, 492, 96
[475  61 489  95]
2
2
2
475, 61, 490, 95
[475  61 489  95]
175, 67, 182, 88
[175  66 183  88]
2
2
2
475, 62, 488, 95
[475  61 488  95]
177, 67, 184, 90
[177  66 185  89]
2
2
2
476, 62, 488, 95
[475  61 489  95]
179, 66, 186, 89
[178  66 186  89]
2
2
2
477, 62, 489, 97
[475  61 488  95]
181, 68, 187, 88
[179  67 188  89]
2
2
1
181, 67, 188, 88
[475  61 488  95]
